### Current non-standard dependecies, panel, param, and matplot lib
#### Install separately: pip install panel, pip install param, pip install -U matplotlib
#### Details these libraries here: http://pyviz.org/

In [ ]:
import numpy as np
#import holoviews as hv
import networkx as nx
import panel as pn
import panel
from panel.layout import Row
import param
import matplotlib.pyplot as plt
panel.extension()
#hv.extension('bokeh')

In [ ]:
from netrd.reconstruction import RandomReconstructor
from netrd.reconstruction import CorrelationMatrixReconstructor
from netrd.reconstruction import FreeEnergyMinimizationReconstructor
from netrd.reconstruction import RegularizedCorrelationMatrixReconstructor
from netrd.reconstruction import PartialCorrelationMatrixReconstructor
from netrd.reconstruction import NaiveMeanFieldReconstructor
from netrd.reconstruction import ThoulessAndersonPalmerReconstructor
from netrd.reconstruction import ExactMeanFieldReconstructor
from netrd.reconstruction import MaximumLikelihoodEstimationReconstructor
from netrd.reconstruction import ConvergentCrossMappingReconstructor
from netrd.reconstruction import MutualInformationMatrixReconstructor
from netrd.reconstruction import OUInferenceReconstructor
from netrd.reconstruction import GraphicalLassoReconstructor
from netrd.reconstruction import NaiveTransferEntropyReconstructor
from netrd.reconstruction import MarchenkoPastur
from netrd.reconstruction import TimeGrangerCausalityReconstructor
from netrd.reconstruction import CorrelationSpanningTree
from netrd.reconstruction import OptimalCausationEntropyReconstructor

In [ ]:
from netrd.distance import JaccardDistance
from netrd.distance import Hamming
from netrd.distance import Frobenius
from netrd.distance import PortraitDivergence
from netrd.distance import IpsenMikhailov
from netrd.distance import HammingIpsenMikhailov
from netrd.distance import ResistancePerturbation
from netrd.distance import NetSimile

In [ ]:
TS = np.loadtxt("../data/synth_4clique_N64_simple.csv",delimiter=',',encoding='utf8')

In [ ]:
options = ['Random','Correlation Matrix','Regularized Correlation Matrix','Partial Correlation Matrix', 
           'Free Energy Minimization','Naive Mean Field','Thouless Anderson Palmer','Exact Mean Field',
           'Maxium Likelihood Estimation','Covergent Cross Mapping','Mutual Information Matrix','OU Inference',
           'Graphical Lasso','Naive Transfer Entropy','Marchenko Pastur','Time Granger Causality', 'Correlation Spanning Tree', 
           'Optimal Causation Entropy']

distanceOptions = ['Jaccard','Hamming','Frobenius','Portrait Divergence',
                   'Ipsen Mikhailov','Hamming Ipsen Mikhailov','Resistance Perturbation','Net Simile']

dist = {'Jaccard': JaccardDistance,
        'Hamming': Hamming,
        'Frobenius': Frobenius,
        'Portrait Divergence': PortraitDivergence,
        'Ipsen Mikhailov': IpsenMikhailov,
        'Hamming Ipsen Mikhailov': HammingIpsenMikhailov,
        'Resistance Perturbation': ResistancePerturbation,
        'Net Simile': NetSimile}

recons = {'Random': RandomReconstructor,
          'Correlation Matrix': CorrelationMatrixReconstructor, 
          'Free Energy Minimization': FreeEnergyMinimizationReconstructor,
          'Regularized Correlation Matrix': RegularizedCorrelationMatrixReconstructor,
          'Partial Correlation Matrix': PartialCorrelationMatrixReconstructor, 
          'Naive Mean Field': NaiveMeanFieldReconstructor,
          'Thouless Anderson Palmer': ThoulessAndersonPalmerReconstructor,
          'Exact Mean Field': ExactMeanFieldReconstructor,
          'Maxium Likelihood Estimation': MaximumLikelihoodEstimationReconstructor,
          'Covergent Cross Mapping':  ConvergentCrossMappingReconstructor ,
          'Mutual Information Matrix':  MutualInformationMatrixReconstructor,
          'OU Inference': OUInferenceReconstructor,
          'Graphical Lasso': GraphicalLassoReconstructor,
          'Naive Transfer Entropy': NaiveTransferEntropyReconstructor,
          'Marchenko Pastur': MarchenkoPastur,
          'Time Granger Causality': TimeGrangerCausalityReconstructor,
          'Correlation Spanning Tree': CorrelationSpanningTree,
          'Optimal Causation Entropy': OptimalCausationEntropyReconstructor
         }

class netrdExplorer(param.Parameterized):
    Graph1 = param.ObjectSelector(default = 'Random' , objects = options)
    Graph2 = param.ObjectSelector(default = 'Random' , objects = options)
    Distance = param.ObjectSelector(default = 'Jaccard', objects = distanceOptions)
    
    @param.depends('Graph1','Graph2','Distance')
    def graphs(self):
        g1Method = recons[self.Graph1]()
        G1 = g1Method.fit(TS, threshold_type='degree', avg_k=15)
        #g1 = hv.Graph.from_networkx(G1, nx.layout.kamada_kawai_layout, weight=None)
        adj = nx.to_numpy_matrix(G1)
        fig1 = plt.figure()
        #ax1 = fig1.add_subplot(111)
        matrix1 = plt.imshow(adj)
        colorbar1 = plt.colorbar(matrix1)
        g1 = fig1
        
        g2Method = recons[self.Graph2]()
        G2 = g2Method.fit(TS, threshold_type='degree', avg_k=15)
        #g2 = hv.Graph.from_networkx(G2, nx.layout.kamada_kawai_layout, weight=None)
        adj = nx.to_numpy_matrix(G2)
        fig2 = plt.figure()
        #ax2 = fig2.add_subplot(111)
        matrix2 = plt.imshow(adj)
        colorbar2 = plt.colorbar(matrix2)
        g2 = fig2
        
        distance = dist[self.Distance]()
        score = distance.dist(G1,G2)
        
        graphs = pn.Row(g1,g2, score)
        
        return graphs
    
    def view(self, **kwargs):
        g1 = self.graphs
        return pn.Row(g1)
    
netrd = netrdExplorer()
pn.Row(netrd.param, netrd.view()).servable()

In [ ]:
#if you get a charmap error re-run the the cell until it does or try restarting the notebook and
#commenting out panel.extenstion() at the top not sure which why it sometimes fail to launch
#uncomment below to run app server
! panel serve --show --port 5008 netrdExplorer.ipynb